Webpage should call  run_lstm_for_input(). This will run the LSTM code on interested input. If returned value from this is 0(he is not likely to leave), print "recommnder as usual"
If returned value is 1 (likely to leave), then call the next module run_rf_for_input(). If returned value is 1 (he does have a purchasing intention), call the recommender

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
#setting up libraries and base configuration

import tensorflow as tf 
tf.compat.v1.set_random_seed(42)
from sklearn import metrics
from sklearn import svm
import os
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout
import pandas as pd
import numpy as np

session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)


Using TensorFlow backend.


Training LSTM first 

In [3]:
def load_datasets(LSTMTrainFile, LSTMTestFile):
    
    LSTMTrain = pd.read_csv(LSTMTrainFile)
    LSTMTest = pd.read_csv(LSTMTestFile)
    
    LSTMTrain.drop(LSTMTrain.columns[[0]], axis = 1, inplace = True)
    LSTMTest.drop(LSTMTest.columns[[0]], axis = 1, inplace = True)
     
    #testDataset.drop(testDataset.columns[[0]], axis = 1, inplace = True)
    
    return LSTMTrain, LSTMTest

In [4]:
def prepare_datasets(LSTMTrain, LSTMTest):
    
    LSTMTrainX = LSTMTrain[LSTMTrain.columns[0:7]]
    LSTMTrainY = LSTMTrain[LSTMTrain.columns[7:8]]
    
    LSTMTestX = LSTMTest[LSTMTest.columns[0:7]]
    LSTMTestY = LSTMTest[LSTMTest.columns[7:8]]
    

    return LSTMTrainX, LSTMTrainY, LSTMTestX, LSTMTestY

In [5]:
def define_lstm_model():
    
    timesteps = 7
    input_dim = 1
    n_classes = 2
    epoch = 30
    batch_size = 16
    n_hidden = 30

    INPUT_ACTION_TYPES = [
        "Administrative",
        "Informational",
        "ProductRelated",
        "Administrative_Duration",
        "Informational_Duration",
        "ProductRelated_Duration",
        "PageValues"
        ]

    LABELS = [
        "False",
        "True"
    ]

    lstm_model = Sequential()
    lstm_model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(1, activation='sigmoid'))
    
    lstm_model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    
    return lstm_model


In [6]:
def running_lstm_model(lstm_model, LSTMTrainX, LSTMTrainY, LSTMTestX, LSTMTestY):
    
    from sklearn.metrics import classification_report
    
    lstm_model.fit(LSTMTrainX, LSTMTrainY, batch_size = 16, epochs = 30)
    predicted = lstm_model.predict(LSTMTestX)
    
    predicted[predicted<0.5]=0
    predicted[predicted>0.5]=1
    
    print(classification_report(LSTMTestY, predicted))
    
    return lstm_model
 

In [7]:
def run_lstm_for_input():
    
    #training the LSTM
    LSTMTrain, LSTMTest = load_datasets("LSTMTrainData.csv", "LSTMTestData.csv")
    LSTMTrainX, LSTMTrainY, LSTMTestX, LSTMTestY = prepare_datasets(LSTMTrain, LSTMTest)

    LSTMTrainX = LSTMTrainX.to_numpy()
    LSTMTestX = LSTMTestX.to_numpy()
    
    LSTMTrainX = LSTMTrainX.reshape(8631, 7,1)
    LSTMTestX = LSTMTestX.reshape(3699,7,1)

    lstm_model = define_lstm_model()
    lstm_model = running_lstm_model(lstm_model, LSTMTrainX, LSTMTrainY, LSTMTestX, LSTMTestY)
    

    
    #runnign LSTM on interested datapoint
    testInput = pd.read_csv("TestInput.csv")

    #doing only the relevant steps of preprocessing
    testInput.ProductRelated[testInput.ProductRelated != 0] = 1
    testInput.ProductRelated[testInput.ProductRelated == 0] = 0

    testInput.Informational[testInput.Informational != 0] = 1
    testInput.Informational[testInput.Informational == 0] = 0

    testInput.Administrative[testInput.Administrative != 0] = 1
    testInput.Administrative[testInput.Administrative == 0] = 0  

    testInput = testInput.loc[:, ["Administrative", "Administrative_Duration", "Informational", "Informational_Duration", "ProductRelated", "ProductRelated_Duration", "PageValues"]]
    
    testInput = np.array(testInput)
    testInput = testInput.reshape(1,7,1)

    testOutput = lstm_model.predict(testInput)
    testOutput[testOutput<0.5]=0
    testOutput[testOutput>0.5]=1

    return testOutput, lstm_model


In [8]:
outputLSTM, lstm_model = run_lstm_for_input()


Epoch 1/30
8631/8631 [==============================] - 9s 1ms/step - loss: 0.3161 - accuracy: 0.8753
Epoch 2/30
8631/8631 [==============================] - 6s 710us/step - loss: 0.2782 - accuracy: 0.8885
Epoch 3/30
8631/8631 [==============================] - 7s 817us/step - loss: 0.2726 - accuracy: 0.8933
Epoch 4/30
8631/8631 [==============================] - 6s 714us/step - loss: 0.2682 - accuracy: 0.8928
Epoch 5/30
8631/8631 [==============================] - 8s 892us/step - loss: 0.2685 - accuracy: 0.8941
Epoch 6/30
8631/8631 [==============================] - 6s 753us/step - loss: 0.2658 - accuracy: 0.8921
Epoch 7/30
8631/8631 [==============================] - 7s 825us/step - loss: 0.2653 - accuracy: 0.8941
Epoch 8/30
8631/8631 [==============================] - 9s 1ms/step - loss: 0.2624 - accuracy: 0.8982
Epoch 9/30
8631/8631 [==============================] - 8s 982us/step - loss: 0.2653 - accuracy: 0.8932
Epoch 10/30
8631/8631 [==============================] - 8s 966us/st

Training RF model

In [9]:
def load_datasets(trainFile, testFile):
    
    import pandas as pd
    trainDataset = pd.read_csv(trainFile)
    testDataset = pd.read_csv(testFile)
    
    trainDataset.drop(trainDataset.columns[[0]], axis = 1, inplace = True)
    testDataset.drop(testDataset.columns[[0]], axis = 1, inplace = True)
    
    return trainDataset, testDataset


In [10]:
#seperating test and train into features and target
def prepare_datasets(trainDataset, testDataset):
    
    trainX = trainDataset[trainDataset.columns[0:15]]
    trainY = trainDataset[trainDataset.columns[15:16]]

    testX = testDataset[testDataset.columns[0:15]]
    testY = testDataset[testDataset.columns[15:16]]

    return trainX, trainY, testX, testY


In [11]:
def randomForest(DataTrainX, DataTrainY, DataTestX, DataTestY):
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import classification_report
    
    rf = RandomForestClassifier(bootstrap= True, max_depth= 80, max_features= 3, min_samples_leaf= 3, min_samples_split= 8, n_estimators= 100)
    rf.fit(DataTrainX, DataTrainY)
    predictedData = rf.predict(DataTestX)
    
    print(classification_report(DataTestY, predictedData))
    
    return rf


In [12]:
def run_rf_for_input():

    #training the dataset
    trainData, testData = load_datasets("RFTrainingData.csv", "RFTestingData.csv")    
    trainDataX, trainDataY, testDataX, testDataY = prepare_datasets(trainData, testData)
    
    rf = randomForest(trainDataX, trainDataY, testDataX, testDataY)
    
    
    #running the model on input instance
    testInput = pd.read_csv("TestInput.csv")
    testInput = testInput.loc[:, ["Month_Nov", "TrafficType_2", "TrafficType_3", "Month_May", "SpecialDay", "TrafficType_13", "OS_3", "TrafficType_1", "Month_Mar", "TrafficType_8", "BounceRates", "ExitRates", "Month_Feb", "ProductRelated_Duration", "PageValues"]]
    testOutput = rf.predict(testInput)
    
    index = np.where(testOutput==1)
    
    print(index)
    
    return testOutput, rf
    
    

In [13]:
testOutput, rf = run_rf_for_input()

              precision    recall  f1-score   support

         0.0       0.95      0.89      0.92      3108
         1.0       0.57      0.76      0.65       591

    accuracy                           0.87      3699
   macro avg       0.76      0.83      0.79      3699
weighted avg       0.89      0.87      0.88      3699

(array([0]),)


In [14]:
##Case 1. LSTM should predict 0 for this customer
##Data Point : 8916,3,142.5,0,0.0,48,1052.255952,0.004347826,0.013043478,0.0,0.0,False,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0


testInput = pd.read_csv("TestInput.csv")

testInput.ProductRelated[testInput.ProductRelated != 0] = 1
testInput.ProductRelated[testInput.ProductRelated == 0] = 0

testInput.Informational[testInput.Informational != 0] = 1
testInput.Informational[testInput.Informational == 0] = 0

testInput.Administrative[testInput.Administrative != 0] = 1
testInput.Administrative[testInput.Administrative == 0] = 0  

testInput = testInput.loc[:, ["Administrative", "Administrative_Duration", "Informational", "Informational_Duration", "ProductRelated", "ProductRelated_Duration", "PageValues"]]
    
testInput = np.array(testInput)
testInput = testInput.reshape(1,7,1)

testOutput = lstm_model.predict(testInput)
testOutput[testOutput<0.5]=0
testOutput[testOutput>0.5]=1

#will be easier for front end if all the outputs are 1d numpy arrays
testOutput = testOutput.flatten()

print(testOutput)


[0.]


In [15]:
##Case 2. RF should return 0 for this customer
##Data Point : 8916,3,142.5,0,0.0,48,1052.255952,0.004347826,0.013043478,0.0,0.0,False,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0

testInput = pd.read_csv("TestInput.csv")

testInput = testInput.loc[:, ["Month_Nov", "TrafficType_2", "TrafficType_3", "Month_May", "SpecialDay", "TrafficType_13", "OS_3", "TrafficType_1", "Month_Mar", "TrafficType_8", "BounceRates", "ExitRates", "Month_Feb", "ProductRelated_Duration", "PageValues"]]

testOutput = rf.predict(testInput)

print(testOutput)
    

[1.]


In [16]:
##Case 3. RF should return 1 for this customer
#Data Point : 6290,1,29.2,0,0.0,0,0.0,0.0,0.066666667,0.0,0.0,True,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1


testInput = pd.read_csv("TestInput.csv")

testInput = testInput.loc[:, ["Month_Nov", "TrafficType_2", "TrafficType_3", "Month_May", "SpecialDay", "TrafficType_13", "OS_3", "TrafficType_1", "Month_Mar", "TrafficType_8", "BounceRates", "ExitRates", "Month_Feb", "ProductRelated_Duration", "PageValues"]]

testOutput = rf.predict(testInput)

print(testOutput)



    

[1.]
